# ANN and Classification in Power Systems with WandB Monitoring

This notebook implements an Artificial Neural Network for power system event detection with comprehensive Weights & Biases (WandB) monitoring for tracking training progress, loss curves, and model performance.

## Installation and Setup

In [ ]:
# Install Weights & Biases
!pip install wandb -qU

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import wandb

In [ ]:
# Login to WandB with your API key
# Replace with your actual API key or use wandb.login() for interactive login
wandb.login(key="use your won API from WandB site. Just copy paste your API code here")

## Data Generation

Generating synthetic data for 10 PMUs with 14 features each:
- Magnitude of current for phases a, b, c
- Magnitude of voltage for phases a, b, c
- Phase angles for voltage and current
- Frequency and Rate of Change of Frequency (ROCOF)

Event labels:
- 0: Normal
- 1: Phase Angle Instability
- 2: Low Frequency Instability
- 3: Voltage Instability
- 4: Large Signal Instability

In [ ]:
# Set seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Define the number of PMUs and features per PMU
num_pm = 10  # Number of PMUs
num_features = 14  # Features as specified

# Define bounds for reasonable power system data (based on common ranges)
bounds = {
    "mag_I": (0, 200), "mag_V": (110, 140),
    "angle_V": (-180, 180), "angle_I": (-180, 180),
    "frequency": (59.5, 60.5), "ROCOF": (-2, 2),
}

# Generate synthetic data
def generate_synthetic_data(num_samples):
    data, labels = [], []
    for _ in range(num_samples):
        sample = []
        for _ in range(num_pm):
            sample.extend(np.random.uniform(bounds["mag_I"][0], bounds["mag_I"][1], 3))
            sample.extend(np.random.uniform(bounds["mag_V"][0], bounds["mag_V"][1], 3))
            sample.extend(np.random.uniform(bounds["angle_V"][0], bounds["angle_V"][1], 3))
            sample.extend(np.random.uniform(bounds["angle_I"][0], bounds["angle_I"][1], 3))
            sample.append(np.random.uniform(bounds["frequency"][0], bounds["frequency"][1]))
            sample.append(np.random.uniform(bounds["ROCOF"][0], bounds["ROCOF"][1]))

        label = np.random.choice([0, 1, 2, 3, 4])
        labels.append(label)
        data.append(sample)

    return np.array(data), np.array(labels)

# Generate 100 samples
X, y = generate_synthetic_data(100)
print(f"Generated data shape: {X.shape}")
print(f"Labels shape: {y.shape}")

## Data Preprocessing

In [ ]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

## Model Definition

Defining a multi-layer ANN with:
- Input layer: 140 features (10 PMUs × 14 features)
- Hidden layers: 128 → 64 → 32 neurons
- Output layer: 5 classes
- Activation: ReLU for hidden layers

In [ ]:
# Define the ANN model
class ANN_Model(nn.Module):
    def __init__(self):
        super(ANN_Model, self).__init__()
        self.fc1 = nn.Linear(num_pm * num_features, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 5)  # 5 output classes

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x

# Initialize model, loss function, and optimizer
model = ANN_Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Convert data to tensors
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.LongTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.LongTensor(y_test)

print("Model architecture:")
print(model)

## Training with WandB Monitoring

Training the model with comprehensive WandB tracking:
- Training loss per epoch
- Validation/test accuracy
- Hyperparameters
- System metrics

In [ ]:
# Initialize WandB run
wandb.init(
    project="power-system-classification",
    name="ann-pmu-classification",
    config={
        "learning_rate": 0.001,
        "architecture": "ANN",
        "dataset": "PMU-Power-System",
        "epochs": 50,
        "num_pmus": num_pm,
        "num_features": num_features,
        "hidden_layers": [128, 64, 32],
        "optimizer": "Adam",
        "batch_size": "full_batch",
        "num_classes": 5,
        "train_samples": len(X_train),
        "test_samples": len(X_test)
    }
)

# Log model architecture
wandb.watch(model, log="all", log_freq=10)

# Train the model
num_epochs = 50
train_losses = []

print("Starting training...")
for epoch in range(num_epochs):
    # Training step
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    train_losses.append(loss.item())

    # Validation step
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test_tensor)
        test_loss = criterion(test_outputs, y_test_tensor)
        _, predicted = torch.max(test_outputs, 1)
        test_accuracy = accuracy_score(y_test, predicted.numpy())

    # Log metrics to WandB
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": loss.item(),
        "test_loss": test_loss.item(),
        "test_accuracy": test_accuracy,
        "learning_rate": optimizer.param_groups[0]['lr']
    })

    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}, Test Accuracy: {test_accuracy:.4f}")

print("\nTraining completed!")

## Model Evaluation and Visualization

In [ ]:
# Final evaluation
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, predicted = torch.max(outputs, 1)
    final_accuracy = accuracy_score(y_test, predicted.numpy())

print(f"\nFinal Test Accuracy: {final_accuracy:.4f}")

# Class labels
class_names = ['Normal', 'Phase Angle', 'Low Freq Instability',
               'Voltage Instability', 'Large Signal Instability']

# Classification report
print("\nClassification Report:")
print(classification_report(y_test, predicted.numpy(), target_names=class_names))

# Log final metrics to WandB
wandb.summary['final_test_accuracy'] = final_accuracy

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Loss over Epochs', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Log plot to WandB
wandb.log({"training_loss_plot": wandb.Image(plt)})
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, predicted.numpy())

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Confusion Matrix', fontsize=14)
plt.tight_layout()

# Log confusion matrix to WandB
wandb.log({"confusion_matrix": wandb.Image(plt)})
plt.show()

In [ ]:
# Create WandB confusion matrix (interactive)
wandb.log({
    "conf_mat": wandb.plot.confusion_matrix(
        probs=None,
        y_true=y_test,
        preds=predicted.numpy(),
        class_names=class_names
    )
})

## Save Model to WandB

In [ ]:
# Save model checkpoint
model_path = "power_system_ann_model.pt"
torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_loss': train_losses[-1],
    'test_accuracy': final_accuracy,
}, model_path)

# Log model to WandB
wandb.save(model_path)
print(f"\nModel saved to {model_path} and uploaded to WandB")

In [ ]:
# Finish WandB run
wandb.finish()
print("\nWandB run finished. Check your WandB dashboard for detailed metrics and visualizations!")

## Summary

This notebook demonstrates:
1. **Data Generation**: Synthetic PMU data with realistic power system values
2. **Model Architecture**: Multi-layer ANN for 5-class classification
3. **WandB Integration**:
   - Real-time loss monitoring
   - Accuracy tracking
   - Hyperparameter logging
   - Model checkpointing
   - Confusion matrix visualization
4. **Performance Metrics**: Classification report and confusion matrix

### Next Steps:
- Visit your WandB dashboard to explore interactive charts
- Compare multiple runs with different hyperparameters
- Share results with your team
- Use WandB Sweeps for hyperparameter optimization